In [73]:
import jsonpickle
import hashlib
import numpy as np

class Account:
    name: str

    def __init__(self, name: str):
        self.name = name

    def __str__(self):
        return "Account(name: {})".format(self.name)

    def __repr__(self):
        return  self.__str__()

class Post:
    id: int
    text: str
    img_path: str

    def __init__(self, id: int, text: str, img_path: str):
        self.id = id
        self.text = text
        self.img_path = text

    def uuid(self) -> str:
        digest = hashlib.sha256()
        digest.update(self.__str__().encode("utf-8"))
        return digest.hexdigest()

    def __str__(self):
        return "Post(id: {}, text: {}, img_path: {})"\
            .format(self.id, self.text, self.img_path)

    def __repr__(self):
        return "Post(id: {}, text: {}, img_path: {})"\
            .format(self.id, self.text, self.img_path)

class Action:
    uuid: str
    account: Account

class PostAction(Action):
    post: Post

    def __init__(self, account: Account, post: Post):
        self.uuid = post.uuid()
        self.account = account
        self.post = post

class RTAction(Action):
    url: str = None

    def __init__(self, account: Account, post: Post):
        self.uuid = post.uuid()
        self.account = account

class SRTAction(Action):
    url: str = None

    def __init__(self, account: Account, post: Post):
        self.uuid = post.uuid()
        self.account = account


In [74]:
from typing import Dict, List

# load accounts
soc_account = Account("@sonsofcryptolabs")
dev_account = Account("@web3dev")

#load lists
cpa_posts = list(map(lambda x: Post(x, "cpa_posts", None), list(np.arange(30))))
mem_posts = list(map(lambda x: Post(x, "mem_posts", None), list(np.arange(30))))
jka_posts = list(map(lambda x: Post(x, "jka_posts", None), list(np.arange(30))))
cpb_posts = list(map(lambda x: Post(x, "cpb_posts", None), list(np.arange(30))))
fre_posts = list(map(lambda x: Post(x, "fre_posts", None), list(np.arange(30))))
jkb_posts = list(map(lambda x: Post(x, "jkb_posts", None), list(np.arange(30))))

def generate_actions() -> List[Action]:
    actions: List = []
    for i in range(0, 30):
        actions.append(PostAction(soc_account,cpa_posts[i]))
        actions.append(PostAction(dev_account,mem_posts[i]))
        actions.append(PostAction(soc_account,jka_posts[i]))
        actions.append(PostAction(dev_account,cpb_posts[i]))
        actions.append(PostAction(soc_account,fre_posts[i]))
        actions.append(PostAction(dev_account,jkb_posts[i]))
        actions.append(RTAction(soc_account,mem_posts[i]))
        actions.append(RTAction(dev_account,cpa_posts[i]))
        actions.append(RTAction(soc_account,cpb_posts[i]))
        actions.append(RTAction(dev_account,jka_posts[i]))
        actions.append(RTAction(soc_account,jkb_posts[i]))
        actions.append(RTAction(dev_account,fre_posts[i]))
        actions.append(SRTAction(soc_account,cpa_posts[i]))
        actions.append(SRTAction(dev_account,mem_posts[i]))
        actions.append(SRTAction(soc_account,jka_posts[i]))
        actions.append(SRTAction(dev_account,cpb_posts[i]))
        actions.append(SRTAction(soc_account,fre_posts[i]))
        actions.append(SRTAction(dev_account,jkb_posts[i]))
    return actions


In [75]:
class Executor:

    def post(self, account: Account, post: Post) -> str:
        print("Posting", account.name, post.id, post.text)
        return "http://twitter.com/" + post.uuid()

    def rt(self, account: Account, url: str, uuid: str):
        print("RTing", account.name, uuid, url)

    def srt(self, account: Account, url: str, uuid: str):
        print("Self RTing", account.name, uuid, url)


def update_actions_url(action: List[Action], uuid: str, url: str):
    for action in actions:
        if uuid == action.uuid and not isinstance(action, PostAction):
            action.url = url

def exec_next_action(executor: Executor, actions: List[Action]) -> List[Action]:
    if len(actions) <= 0:
        return actions

    action = actions.pop(0)

    if isinstance(action, PostAction):
        url = executor.post(action.account, action.post)
        update_actions_url(actions, action.uuid, url)
    elif isinstance(action, RTAction):
        executor.rt(action.account, action.url, action.uuid)
    elif isinstance(action, SRTAction):
        executor.srt(action.account, action.url, action.uuid)
    else:
        print("Unexpected object type", action)

    return actions

def write_to_file(data: bytes, path: str):
    with open(path, "w") as file:
        file.write(data)

def read_from_file(path: str) -> bytes:
    try:
        with open(path, "r") as file:
            return file.read()
    except:
        return None

In [76]:
ACTIONS_PATH = ".run/actions.json"
executor = Executor()

# restore or generate actions if needed
actions_data = read_from_file(ACTIONS_PATH)
if actions_data is not None:
    actions = jsonpickle.decode(actions_data)
else:
    actions = generate_actions()
    write_to_file(jsonpickle.encode(actions), ACTIONS_PATH)

# executes action & write new state to disk
print("Actions count:", len(actions))
if len(actions) > 0:
    actions = exec_next_action(executor, actions)
    write_to_file(jsonpickle.encode(actions), ACTIONS_PATH)

Actions count: 491
Self RTing @web3dev b3f6003e947c98d1e5924ac9f0a88457afe94e8ac37a079d58b713125d59499a http://twitter.com/b3f6003e947c98d1e5924ac9f0a88457afe94e8ac37a079d58b713125d59499a


In [ ]:
class Executor:

    def post(self, account: Account, post: Post) -> str:
        print("Posting", account.name, post.id, post.text)
        return "http://twitter.com/" + post.uuid()

    def rt(self, account: Account, url: str, uuid: str):
        print("RTing", account.name, uuid, url)

    def srt(self, account: Account, url: str, uuid: str):
        print("Self RTing", account.name, uuid, url)

In [77]:
# restore actions from disc
# if empty generate actions list
# execute


In [78]:
# 11:00:00 AM	08:00:00 AM	05:00:00 AM	02:00:00 AM
# 12:00:00 PM	09:00:00 AM	06:00:00 AM	03:00:00 AM		CCA
# 01:00:00 PM	10:00:00 AM	07:00:00 AM	04:00:00 AM			meme
# 02:00:00 PM	11:00:00 AM	08:00:00 AM	05:00:00 AM		joke1
# 03:00:00 PM	12:00:00 PM	09:00:00 AM	06:00:00 AM			CCB
# 04:00:00 PM	01:00:00 PM	10:00:00 AM	07:00:00 AM		freedom
# 05:00:00 PM	02:00:00 PM	11:00:00 AM	08:00:00 AM			joke2
# 06:00:00 PM	03:00:00 PM	12:00:00 PM	09:00:00 AM		meme RT
# 07:00:00 PM	04:00:00 PM	01:00:00 PM	10:00:00 AM			CCA RT
# 08:00:00 PM	05:00:00 PM	02:00:00 PM	11:00:00 AM		CCB RT
# 09:00:00 PM	06:00:00 PM	03:00:00 PM	12:00:00 PM			joke1 RT
# 10:00:00 PM	07:00:00 PM	04:00:00 PM	01:00:00 PM		joke2 RT
# 11:00:00 PM	08:00:00 PM	05:00:00 PM	02:00:00 PM			freedom RT
# 12:00:00 AM	09:00:00 PM	06:00:00 PM	03:00:00 PM		CCA SRT
# 01:00:00 AM	10:00:00 PM	07:00:00 PM	04:00:00 PM			meme SRT
# 02:00:00 AM	11:00:00 PM	08:00:00 PM	05:00:00 PM		joke1 SRT
# 03:00:00 AM	12:00:00 AM	09:00:00 PM	06:00:00 PM			CCB SRT
# 04:00:00 AM	01:00:00 AM	10:00:00 PM	07:00:00 PM		freedom SRT
# 05:00:00 AM	02:00:00 AM	11:00:00 PM	08:00:00 PM			joke2 SRT